In [81]:
import sqlite3
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar

In [82]:
N = norm.cdf


In [8]:
def read_from_db():
    conn = sqlite3.connect('../../options_database.db')
    df = pd.read_sql_query("SELECT * FROM options_table", conn)
    conn.close()
    return df

In [9]:
df = read_from_db()

In [41]:
len(df[(df.current_time == '2023-01-29 11:24:46.988931')])

610

In [206]:
selected_put = (df[(df.full_name == 'ETH-27JAN23-1500')][df.current_time > '2023-01-04'][df.type == 'put'])
selected_put['timedelta'] = selected_put.apply(lambda x: get_timedelta_in_years(datetime.fromisoformat(x['expiry']) - datetime.fromisoformat(x['current_time'])), axis=1)


/var/folders/8t/1dvfc69d0l3137hz8kyzv__80000gn/T/ipykernel_9167/828420701.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_put = (df[(df.full_name == 'ETH-27JAN23-1500')][df.current_time > '2023-01-04'][df.type == 'put'])


In [207]:
# selected_put

In [208]:
# print((df.full_name.drop_duplicates()).to_numpy())

In [209]:
def delta_BS_put(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return N(d1) - 1

def delta_BS_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return N(d1)

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
#     print(d1, d2)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T)* N(-d2) - S * N(-d1)

In [210]:
BS_PUT(np.array([1000]), np.array([1000]), np.array([1]), np.array([0]), np.array([1]))

array([382.92492255])

In [211]:
selected_put['mid_price'] = selected_put['mid_price'].fillna(0)
selected_put['ask_price'] = selected_put['ask_price'].fillna(0)
selected_put['bid_price'] = selected_put['bid_price'].fillna(0)

In [212]:
# selected_put['delta']

In [213]:
selected_put.iloc[1]

index                                      222
full_name                     ETH-27JAN23-1500
type                                       put
open_interest                            907.0
expiry              2023-01-27 00:00:00.000000
strike                                    1500
bid_price                                  0.0
bid_IV                                     NaN
ask_price                                  0.0
ask_IV                                     NaN
mid_price                                  0.0
mid_IV                                     NaN
underlying_price                       1211.16
current_time        2023-01-04 00:33:40.899854
timedelta                             0.062907
Name: 17834, dtype: object

In [214]:
from datetime import datetime

In [215]:
def get_timedelta_in_years(timedelta):
    d = timedelta.days / (365.25)
    s = timedelta.seconds / (365.25*24*60*60)
    return d + s

In [216]:
# datetime.strptime(selected_put.iloc[5]['expiry'], "%Y %m %d %I:%M%p")
get_timedelta_in_years(datetime.fromisoformat(selected_put.iloc[5]['expiry']) - datetime.fromisoformat(selected_put.iloc[5]['current_time']))


0.06271452835450098

In [217]:
filtered = selected_put.loc[(selected_put.bid_IV > 0) & (selected_put.bid_IV < 10) & 
                            (selected_put.ask_IV > 0) & (selected_put.ask_IV < 10) &
                             (selected_put.timedelta > 0)]

In [218]:
filtered['strike'] = filtered['strike'].astype(float)

/var/folders/8t/1dvfc69d0l3137hz8kyzv__80000gn/T/ipykernel_9167/1541335339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['strike'] = filtered['strike'].astype(float)


In [219]:
filtered['BS_delta'] = delta_BS_put(filtered.underlying_price, filtered.strike, filtered.timedelta, 0, filtered.mid_IV)



/var/folders/8t/1dvfc69d0l3137hz8kyzv__80000gn/T/ipykernel_9167/3576649484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['BS_delta'] = delta_BS_put(filtered.underlying_price, filtered.strike, filtered.timedelta, 0, filtered.mid_IV)


In [220]:
filtered

,index,full_name,type,open_interest,expiry,strike,bid_price,bid_IV,ask_price,ask_IV,mid_price,mid_IV,underlying_price,current_time,timedelta,BS_delta
352930,196,ETH-27JAN23-1500,put,939.0,2023-01-27 00:00:00.000000,1500.0,0.0800,0.500767,0.0840,0.556310,0.08200,0.528761,1412.75,2023-01-12 03:56:21.573827,0.040618,-0.694645
353410,196,ETH-27JAN23-1500,put,939.0,2023-01-27 00:00:00.000000,1500.0,0.0840,0.504221,0.0865,0.539923,0.08525,0.522183,1406.03,2023-01-12 04:21:32.882211,0.040570,-0.713114
353892,196,ETH-27JAN23-1500,put,939.0,2023-01-27 00:00:00.000000,1500.0,0.0870,0.506860,0.0890,0.535997,0.08800,0.521511,1401.10,2023-01-12 04:46:45.315404,0.040523,-0.724820
354374,196,ETH-27JAN23-1500,put,939.0,2023-01-27 00:00:00.000000,1500.0,0.0850,0.518555,0.0875,0.553982,0.08625,0.536370,1405.95,2023-01-12 05:11:56.964737,0.040475,-0.707504
354860,196,ETH-27JAN23-1500,put,939.0,2023-01-27 00:00:00.000000,1500.0,0.0855,0.520085,0.0875,0.548557,0.08650,0.534388,1405.22,2023-01-12 05:37:09.298762,0.040427,-0.710127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862270,468,ETH-27JAN23-1500,put,17871.0,2023-01-27 00:00:00.000000,1500.0,0.0005,0.857983,0.0010,0.985001,0.00075,0.927465,1595.90,2023-01-26 20:38:19.837640,0.000383,-0.000311
862904,468,ETH-27JAN23-1500,put,17872.0,2023-01-27 00:00:00.000000,1500.0,0.0005,0.854016,0.0010,0.981221,0.00075,0.923580,1593.58,2023-01-26 21:03:38.708310,0.000335,-0.000167
863538,468,ETH-27JAN23-1500,put,17872.0,2023-01-27 00:00:00.000000,1500.0,0.0005,0.923439,0.0010,1.058329,0.00075,0.997277,1601.24,2023-01-26 21:28:56.586539,0.000287,-0.000054
864172,468,ETH-27JAN23-1500,put,17873.0,2023-01-27 00:00:00.000000,1500.0,0.0005,0.927209,0.0010,1.063142,0.00075,1.001604,1599.76,2023-01-26 21:54:16.676830,0.000239,-0.000016


In [241]:
initial_option_price = filtered['mid_price'].iloc[0] * filtered['underlying_price'].iloc[0]
initial_option_price

115.8455

In [222]:
current_delta, loss = 0, 0

for row in filtered.values:
    loss += (row[-1] - current_delta) * row[-4]
    # print(row[-1] - current_delta)
    current_delta = row[-1]
    # print(loss)
loss

117.85719401756214

In [224]:
# filtered